In [1]:
level = 'cntyvtd'
district_type = 'cd'
proj_id = 'cmat-315920'
root_path = '/home/jupyter'

import os, pathlib, google, numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt, plotly.express as px
from google.cloud import aiplatform, bigquery
try:
    from google.cloud.bigquery_storage import BigQueryReadClient
except:
    os.system('pip install --upgrade google-cloud-bigquery-storage')
    from google.cloud.bigquery_storage import BigQueryReadClient

cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient   = bigquery.Client(credentials=cred, project=proj)
root_path = pathlib.Path('/home/jupyter')
tbl = f'{proj_id}.redistricting_data.cook_plans_TX_2020_{level}_{district_type}_stacked'
df = bqclient.query(f'select * from {tbl}').result().to_dataframe().rename(columns={'geoid':district_type})
geo = gpd.GeoSeries.from_wkt(df['polygon'], crs='EPSG:4326').simplify(0.05).buffer(0)
gdf = gpd.GeoDataFrame(df, geometry=geo)
gdf['color'] = gdf['color'].astype(str)
gdf['locations'] = gdf.index

NotFound: 404 Not found: Table cmat-315920:redistricting_data.cook_plans_TX_2020_cntyvtd_cd_stacked was not found in location US

(job ID: 269df8e8-ff2d-41f5-9869-4ec0e9424ef1)

                            -----Query Job SQL Follows-----                            

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:select * from cmat-315920.redistricting_data.cook_plans_TX_2020_cntyvtd_cd_stacked
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |

In [ ]:
opts = {
    'locations' : 'locations',
    'color' : "color",
    'color_discrete_sequence' : px.colors.qualitative.T10,
    'mapbox_style' : 'carto-positron',
    'opacity' : .4,
    'center' : {'lon': -100.05, 'lat': 31.0},
    'hover_data' : {'color':False, 'plan': False, 'locations':False,
                    'cd': True, 'total_pop': True, 'polsby_popper': True}
}

In [2]:
def make_images(animate=False, num_steps=1000000):
    num_steps = min(num_steps, df['plan'].max())
    if animate:
        opts['height'] = 1.13 * opts['width']
        X = gdf.query(f'plan <= {num_steps}')
        fig = px.choropleth_mapbox(X, geojson=X.geometry, **opts, animation_frame='plan', animation_group=X[district_type])
        fig.update_geos(fitbounds="locations", visible=False)
        fig.update(layout_showlegend=False)
        fig.write_html(root_path / 'images/recomb.html', auto_play=False)#, include_plotlyjs=True)
    else:
        opts['height'] = 0.97 * opts['width']
        for k in range(num_steps+1):
            print(k)
            X = gdf.query(f'plan == {k}')
            fig = px.choropleth_mapbox(X, geojson=X.geometry, **opts)
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update(layout_showlegend=False)
            dig = len(str(num_steps))
            fig.write_html(root_path / f'images/html/recomb_{str(k).rjust(dig,"0")}.html')
            fig.write_image(root_path / f'images/png/recomb_{str(k).rjust(dig,"0")}.png')
            fig.write_image(root_path / f'images/svg/recomb_{str(k).rjust(dig,"0")}.svg')
            fig.write_image(root_path / f'images/eps/recomb_{str(k).rjust(dig,"0")}.eps')
    return fig

# opts['zoom'] = 3.0; opts['width'] = 310
opts['zoom'] = 4.0; opts['width'] = 460
# opts['zoom'] = 5.0; opts['width'] = 760

fig = make_images(animate=False, num_steps=20)
fig.show()
# fig = make_images(animate=True, num_steps=20)
# fig.show()

NameError: name 'opts' is not defined

In [5]:
fig.write_image?

Signature: fig.write_image(*args, **kwargs)
Docstring:
Convert a figure to a static image and write it to a file or writeable
object

Parameters
----------
file: str or writeable
    A string representing a local file path or a writeable object
    (e.g. a pathlib.Path object or an open file descriptor)

format: str or None
    The desired image format. One of
      - 'png'
      - 'jpg' or 'jpeg'
      - 'webp'
      - 'svg'
      - 'pdf'
      - 'eps' (Requires the poppler library to be installed)

    If not specified and `file` is a string then this will default to the
    file extension. If not specified and `file` is not a string then this
    will default to `plotly.io.config.default_format`

width: int or None
    The width of the exported image in layout pixels. If the `scale`
    property is 1.0, this will also be the width of the exported image
    in physical pixels.

    If not specified, will default to `plotly.io.config.default_width`

height: int or None
    The height 